In [1]:
import json
import os
import re
import shutil

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFilter

In [2]:
annotations_dir = '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/ann/'
images_dir = '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/img/'

output_dir = '/Users/erikstorrs/Documents/ding/cell_segmentation/data/clustered_dapi_annotated'

In [3]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# do masks

In [4]:
fps = [os.path.join(annotations_dir, fp) for fp in os.listdir(annotations_dir)]
len(fps), fps[:5]

(20,
 ['/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/ann/3.png.json',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/ann/2.png.json',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/ann/11.png.json',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/ann/10.png.json',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/ann/9.png.json'])

In [5]:
def get_masks(vgg_annotator_json_fp):
    d = json.load(open(vgg_annotator_json_fp))
    
    masks = []
    width = d['size']['width']
    height = d['size']['height']
    
    for obj in d['objects']:
        pts = [(x, y) for x, y in obj['points']['exterior']]
        
        img = Image.new('L', (width, height), 0)
        ImageDraw.Draw(img).polygon(pts, outline=1, fill=1)
        mask = np.array(img).astype(bool)
        
        masks.append(mask)
        
    return masks

In [6]:
has_masks = set()
for fp in fps:
    sample = fp.split('/')[-1].split('.')[0]
    
    masks = get_masks(fp)
    
    if masks:
        if not os.path.isdir(os.path.join(output_dir, sample)):
            os.mkdir(os.path.join(output_dir, sample))
            os.mkdir(os.path.join(output_dir, sample, 'masks'))

        for i, mask in enumerate(masks):
            cv2.imwrite(os.path.join(output_dir, sample, 'masks', f'mask_{i}.png'), mask.astype(np.uint8) * 255)          
        has_masks.add(sample)

## do images

In [7]:
fps = [os.path.join(images_dir, fp) for fp in os.listdir(images_dir)]
len(fps), fps[:5]

(20,
 ['/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/img/8.png',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/img/9.png',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/img/14.png',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/img/15.png',
  '/Users/erikstorrs/Documents/ding/cell_segmentation/data/in_house_nuclei/img/17.png'])

In [8]:
for fp in fps:
    sample = fp.split('/')[-1].replace('.png', '')
    if sample in has_masks:
        if not os.path.isdir(os.path.join(output_dir, sample, 'images')):
            os.mkdir(os.path.join(output_dir, sample, 'images'))
        shutil.copyfile(fp, os.path.join(output_dir, sample, 'images', f'{sample}.png'))